# Изучение морфологической разметки

## Токенизация датасета

In [1]:
import pandas as pd

In [2]:
# Выгрузим датасет
corpora_path = '/kaggle/input/kinoart-new-reviews/corpus.csv'
df = pd.read_csv(corpora_path, sep=',', encoding='utf-8')
df.head()

,title,date,author_name,profile_link,text
0,КТО-НИБУДЬ ВИДЕЛ СЕРГЕЯ ДОБРОТВОРСКОГО(?): КРИ...,01.02.21,Зинаида Пронченко,https://kinoart.ru/search?author=2,До российского проката добралась картина Ангел...
1,ДУША ПРИПОМИНАЕТ: «ГЕРДА» НАТАЛЬИ КУДРЯШОВОЙ,18.10.21,Ксения Ильина,https://kinoart.ru/search?author=313,"В прокате «Герда», вторая полнометражная режис..."
2,"«ВСЕМ ПЛОХО, ВСЕ НЕСЧАСТНЫ, И НЕТ СПАСЕНИЯ»: В...",13.08.20,Елена Стишова,https://kinoart.ru/search?author=32,"Проект ИК о постсоветском кино «Пролегомены», ..."
3,"ЛЮК БЕССОН, ГЕРОИНЯ И НАСИЛИЕ. О ФИЛЬМЕ «НИКИТА»",18.03.19,Михаил Трофименков,https://kinoart.ru/search?author=63,Французскому режиссеру Люку Бессону исполнилос...
4,«БРАТ И СЕСТРА»: В ПОИСКАХ ПРОЩЕНИЯ,19.09.22,Яна Телова,https://kinoart.ru/search?author=437,"В прокат вышел фильм «Брат и сестра», последня..."


В дальнейшем будем использовать текст страницы. Нужно будет токенизировать текст по словам. Будем использовать open-source библиотеку SpaCy.

In [3]:
texts = df['text'].values

In [4]:
from spacy.lang.ru import Russian

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
tokenizer = Russian()
doc = tokenizer(texts[0])
words = [token.text for token in doc]
print(words)

['До', 'российского', 'проката', 'добралась', 'картина', 'Ангелины', 'Никоновой', '«', 'Кто', '-', 'нибудь', 'видел', 'мою', 'девчонку', '?', '»', ',', 'которая', 'вызвала', 'негодование', 'среди', 'кинокритиков', 'на', 'прошлогоднем', '«', 'Кинотавре', '»', '.', 'Фильм', 'снят', 'по', 'одноименным', 'мемуарам', 'Карины', 'Добротворской', ',', 'описывающим', 'ее', 'отношения', 'с', 'известным', 'кинокритиком', 'Сергеем', 'Добротворским', '.', 'Зинаида', 'Пронченко', 'анализирует', 'картину', 'и', 'объясняет', ',', 'почему', 'экранизация', 'вышла', 'слабой', '.', '\n', '«', 'Кто', '-', 'нибудь', 'видел', 'мою', 'девчонку', '»', 'снят', 'по', 'одноименному', 'автобиографическому', 'роману', 'Карины', 'Добротворской', '.', 'Возвращаться', 'к', 'книге', 'и', 'сравнивать', 'литературный', 'источник', 'с', 'его', 'экранной', 'версией', 'мы', 'здесь', 'не', 'будем', '.', 'Ведь', 'не', 'разбирают', 'же', '«', 'Анну', 'Каренину', '»', 'Толстого', 'каждый', 'раз', ',', 'когда', 'кто', '-', 'то',

Заметно, что токенизация работает довольно хорошо и качественно. 

## Морфологический анализ

### Выбор морфопарсера

Существует множество решений, основные из которых были представлены в [MorphRuEval 2017](https://github.com/dialogue-evaluation/morphoRuEval-2017) и [GramEval 2020](https://github.com/dialogue-evaluation/GramEval2020).

* [qbic](https://github.com/DanAnastasyev/GramEval2020) - ELMO и BERT модели, которые достигли топовых метрик на соревновании. Но тем не менее, будет наблюдаться множество галлюцинаций, например при лемматизации. "потерь" -> "потеь"
* [rnnmorph](https://github.com/IlyaGusev/rnnmorph) - рекуррентная нейронная сеть, у которой для самопроверки и уменьшения галлюцинаций есть словарь. Засчет этого данная модель будет достигать наилучших результатов. 
* [pymorphy2](https://github.com/pymorphy2/pymorphy2) - статистический словарный анализатор, который не использует контекст и работает быстро. Но у него очень низкое качество засчет жадного выбора наиболее вероятного варианта.

Поэтому будем использовать rnnmorph, так как он достаточно качественный и простой в архитектуре.

### RNN Morph

In [ ]:
!pip install tensorflow --ignore-installed

In [ ]:
!pip3 install rnnmorph

In [8]:
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language="ru")

forms = predictor.predict(words)

<frozen importlib._bootstrap>:241: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: closure mismatch, requested ('self', 'step_function'), but source function had ()
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 4s 4s/step


In [ ]:
forms

Теперь распарсим все слова во всех текстах и сохраним данную информацию вместе с текстами, для удобного доступа. 

In [10]:
from tqdm import tqdm

In [ ]:
all_docs = [tokenizer(doc) for doc in tqdm(texts)]
all_words = [[token.text for token in doc] for doc in tqdm(all_docs)]

In [ ]:
rnn_morph_tags = [predictor.predict(words) for words in tqdm(all_words)]

In [ ]:
config = {"punct_chars": [".", "?", "!", "。"]}
sentencizer = tokenizer.add_pipe("sentencizer", config=config)

In [31]:
import numpy as np

def sample(texts, tokenizer, predictor):
    text = np.random.choice(texts)

    tokens = tokenizer(text)
    sentences = [sent.text for sent in tokens.sents]
    sentence = np.random.choice(sentences)
    print(sentence)
    sent = tokenizer(str(sentence))
    words = [token.text for token in sent]
    
    predictions = predictor.predict(words)
    return sentence, predictions

#### Пример сэмплированного текста

In [32]:
sample(texts, tokenizer, predictor)

И одновременно — объяснение того, что собирается сделать Тодд Хейнс.
1/1 [==============================] - 0s 42ms/step


('И одновременно — объяснение того, что собирается сделать Тодд Хейнс.',
 [<normal_form=и; word=И; pos=CONJ; tag=_; score=0.8091>,
  <normal_form=одновременно; word=одновременно; pos=ADV; tag=Degree=Pos; score=0.9375>,
  <normal_form=—; word=—; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=объяснение; word=объяснение; pos=NOUN; tag=Case=Nom|Gender=Neut|Number=Sing; score=0.9997>,
  <normal_form=тот; word=того; pos=PRON; tag=Case=Gen|Gender=Neut|Number=Sing; score=0.9991>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=что; word=что; pos=PRON; tag=Case=Nom|Gender=Neut|Number=Sing; score=0.5441>,
  <normal_form=собираться; word=собирается; pos=VERB; tag=Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Mid; score=1.0000>,
  <normal_form=сделать; word=сделать; pos=VERB; tag=VerbForm=Inf; score=1.0000>,
  <normal_form=тодда; word=Тодд; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Plur; score=0.3173>,
  <normal_form=хейнс; word=Хейнс; pos=NOUN; tag=Case

Видно, что в основном морфопарсер справился со своей задачей. Но в то же время есть несколько существенных недочетов. 

* "Тодд" слемматизировалось до "тодда", что является ошибочным. Также неправильно определился женский род, множественное число и падеж.

    ``` <normal_form=тодда; word=Тодд; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Plur; score=0.3173>```
* В остальном ошибок не наблюдается. 

In [35]:
sample(texts, tokenizer, predictor)


Несмотря на киберпанковскую мифологию общества, в котором будущее и настоящее совпадают, фантастический дизайн фильма кажется удивительно архаичным.
1/1 [==============================] - 0s 51ms/step


('\nНесмотря на киберпанковскую мифологию общества, в котором будущее и настоящее совпадают, фантастический дизайн фильма кажется удивительно архаичным.',
 [<normal_form=
  ; word=
  ; pos=CONJ; tag=_; score=0.9918>,
  <normal_form=несмотря; word=Несмотря; pos=ADV; tag=Degree=Pos; score=0.9625>,
  <normal_form=на; word=на; pos=ADP; tag=_; score=1.0000>,
  <normal_form=киберпанковский; word=киберпанковскую; pos=ADJ; tag=Case=Acc|Degree=Pos|Gender=Fem|Number=Sing; score=0.9999>,
  <normal_form=мифология; word=мифологию; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=общество; word=общества; pos=NOUN; tag=Case=Gen|Gender=Neut|Number=Sing; score=0.9959>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=который; word=котором; pos=DET; tag=Case=Loc|Gender=Neut|Number=Sing; score=0.9853>,
  <normal_form=будущее; word=будущее; pos=NOUN; tag=Case=Nom|Gender=Neut|Number=Sing; score=0.9

В примере выше, 2 слова: "будущее" и "настоящее"- играющие одинковую роль в предложении, распарсились по разному. Модель правильно разобрала первое слово, но определила второе как прилагательное.
```
<normal_form=будущее; word=будущее; pos=NOUN; tag=Case=Nom|Gender=Neut|Number=Sing; score=0.9485>
<normal_form=настоящий; word=настоящее; pos=ADJ; tag=Case=Nom|Degree=Pos|Gender=Neut|Number=Sing; score=0.6208>
```

In [52]:
sample(texts, tokenizer, predictor)

Тут сказывается не только емкость истории, но и мастеровитость режиссера: новеллу делал Роберт Вэлли, режиссера клипов Gorillaz и пары эпизодов в анимационных сериалах «Эон Флакс» и «Трон: Восстание».
1/1 [==============================] - 0s 71ms/step


('Тут сказывается не только емкость истории, но и мастеровитость режиссера: новеллу делал Роберт Вэлли, режиссера клипов Gorillaz и пары эпизодов в анимационных сериалах «Эон Флакс» и «Трон: Восстание».',
 [<normal_form=тут; word=Тут; pos=ADV; tag=Degree=Pos; score=0.9997>,
  <normal_form=сказываться; word=сказывается; pos=VERB; tag=Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Mid; score=1.0000>,
  <normal_form=не; word=не; pos=PART; tag=_; score=1.0000>,
  <normal_form=только; word=только; pos=ADV; tag=Degree=Pos; score=0.9997>,
  <normal_form=ёмкость; word=емкость; pos=NOUN; tag=Case=Nom|Gender=Fem|Number=Sing; score=0.9890>,
  <normal_form=история; word=истории; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Sing; score=0.9941>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=но; word=но; pos=CONJ; tag=_; score=1.0000>,
  <normal_form=и; word=и; pos=PART; tag=_; score=0.9988>,
  <normal_form=мастеровитость; word=мастеровитость; pos=NOUN; tag=Cas

Можем заметить, что данный парсер причисляет неизвестные символы и слова на англиском ("Gorrillaz" в даном случае) к пунктуации. 
```
<normal_form=gorillaz; word=Gorillaz; pos=PUNCT; tag=_; score=1.0000>
```

#### Попробуем насэмплировать примеры, на которых модель ошибается.

In [72]:
sample(texts, tokenizer, predictor)

Dark Souls — хардкорная эшкн-РПГ, в которой игроку предстоит умереть не раз и не два.
1/1 [==============================] - 0s 47ms/step


('Dark Souls — хардкорная эшкн-РПГ, в которой игроку предстоит умереть не раз и не два.',
 [<normal_form=dark; word=Dark; pos=PUNCT; tag=_; score=0.8634>,
  <normal_form=souls; word=Souls; pos=VERB; tag=Mood=Imp|Number=Sing|Person=2|VerbForm=Fin|Voice=Act; score=0.2086>,
  <normal_form=—; word=—; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=хардкорный; word=хардкорная; pos=ADJ; tag=Case=Nom|Degree=Pos|Gender=Fem|Number=Sing; score=0.9982>,
  <normal_form=эшкн; word=эшкн; pos=NOUN; tag=Case=Nom|Gender=Fem|Number=Sing; score=0.9910>,
  <normal_form=-; word=-; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=рпг; word=РПГ; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.7992>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=который; word=которой; pos=DET; tag=Case=Loc|Gender=Fem|Number=Sing; score=0.9993>,
  <normal_form=игрок; word=игроку; pos=NOUN; tag=Case=Dat|Gender=Masc|Number=Sing; score=0.9

Тут заметны слудеющие интересные недочеты: 
* ```<normal_form=souls; word=Souls; pos=VERB; tag=Mood=Imp|Number=Sing|Person=2|VerbForm=Fin|Voice=Act; score=0.2086>```

    Модель видимо иногда срабатывает для слов на английском, но качество там низкое. Скорее всего это результат галлюцинации, так как в словарях нет значения "souls", которое бы было глаголом. Можно сделать предположение, что в отличие от русского языка, модель не использует словарь для англиских слов.
*  ```<normal_form=эшкн; word=эшкн; pos=NOUN; tag=Case=Nom|Gender=Fem|Number=Sing; score=0.9910>,
   <normal_form=-; word=-; pos=PUNCT; tag=_; score=1.0000>,
   <normal_form=рпг; word=РПГ; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.7992>,```
   
  Тут уже видны артефакты выбранного токенизатора, который разбивает сложные слова с дефисом.

In [88]:
sample(texts, tokenizer, predictor)


Аккурат год назад Райан Мёрфи снял сказку о том, что все могло быть иначе — хотя бы в кузнице мечт Голливуде, который то ли питается социальным запросом, то ли его формирует, — поди разберись, много на себя взяло кино или все-таки поместится еще тюк-другой.
1/1 [==============================] - 0s 94ms/step


('\nАккурат год назад Райан Мёрфи снял сказку о том, что все могло быть иначе — хотя бы в кузнице мечт Голливуде, который то ли питается социальным запросом, то ли его формирует, — поди разберись, много на себя взяло кино или все-таки поместится еще тюк-другой.',
 [<normal_form=
  ; word=
  ; pos=CONJ; tag=_; score=0.9939>,
  <normal_form=аккурат; word=Аккурат; pos=ADV; tag=Degree=Pos; score=0.9947>,
  <normal_form=год; word=год; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Sing; score=0.8504>,
  <normal_form=назад; word=назад; pos=ADP; tag=_; score=0.9999>,
  <normal_form=райан; word=Райан; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9998>,
  <normal_form=мёрфь; word=Мёрфи; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Sing; score=0.9822>,
  <normal_form=снять; word=снял; pos=VERB; tag=Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act; score=1.0000>,
  <normal_form=сказка; word=сказку; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Sing; score=0.9999>,
  <normal_form=о;

Тут интересно рассмотреть такую часть предложения: "— поди разберись, много на себя взяло кино или все-таки поместится еще тюк-другой." В данном случае "поди" распарсилось неправильно, а именно - как наречие вместо частицы. Тут могло повлиять контекстное обучение, так как слово стоит перед глаголом, что свойственно наречиям. 
```
<normal_form=поди; word=поди; pos=ADV; tag=Degree=Pos; score=0.2773>
```

In [93]:
sample(texts, tokenizer, predictor)

сигнализируют не только обломки знакомых эстетик, но и остроумный неймдроппинг.
1/1 [==============================] - 0s 41ms/step


('сигнализируют не только обломки знакомых эстетик, но и остроумный неймдроппинг.',
 [<normal_form=сигнализировать; word=сигнализируют; pos=VERB; tag=Mood=Ind|Number=Plur|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act; score=1.0000>,
  <normal_form=не; word=не; pos=PART; tag=_; score=0.9999>,
  <normal_form=только; word=только; pos=ADV; tag=Degree=Pos; score=0.9973>,
  <normal_form=обломок; word=обломки; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Plur; score=0.8311>,
  <normal_form=знакомый; word=знакомых; pos=ADJ; tag=Case=Gen|Degree=Pos|Number=Plur; score=0.9210>,
  <normal_form=эстетика; word=эстетик; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Plur; score=0.9778>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=но; word=но; pos=CONJ; tag=_; score=1.0000>,
  <normal_form=и; word=и; pos=PART; tag=_; score=0.9748>,
  <normal_form=остроумный; word=остроумный; pos=ADJ; tag=Case=Acc|Degree=Pos|Gender=Masc|Number=Sing; score=0.9387>,
  <normal_form=неймдроппинг; word=

Тут у слов "обломки" и "неймдроппинг" неправильно определился падеж: парсер поставил винительный (Acc) вместо именительного (Pos). 
```
 <normal_form=обломок; word=обломки; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Plur; score=0.8311>
 <normal_form=неймдроппинг; word=неймдроппинг; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Sing; score=0.9371>
```

### Примеры омонимии

#### Рассмотрим лексическую (полную омонимию)

Можно конечно рассмотреть пример с лексической омонимией, но данный морфологический парсер не позволяет различать данные случаи, так как у них будут совпадать леммы и прочие параметры тэгов. 

In [94]:
# Функция, которая сэмплит предложение с вхождением данного слова
def sample_with_word(texts, tokenizer, predictor, find_word):
    found_texts = [t for t in texts if find_word in t]
    sentences = [s for t in found_texts for s in tokenizer(t).sents]
    found_sentences = [s for s in sentences if find_word in s.text]
    sentence = np.random.choice(found_sentences)

    sent = tokenizer(sentence.text)
    words = [token.text for token in sent]

    predictions = predictor.predict(words)
    return sentence.text, predictions

##### Топить

In [95]:
sample_with_word(texts, tokenizer, predictor, 'топить')

/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


1/1 [==============================] - 0s 161ms/step


('Своей похотью, своей жаждой жизни Яна способна испепелить горные долины, растопить снежные горные пики, но выбирает, чтобы огонь шел за ней, вглубь ее души, в извилины ее головы, то затухал, то занимался вновь.',
 [<normal_form=свой; word=Своей; pos=DET; tag=Case=Ins|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=похоть; word=похотью; pos=NOUN; tag=Case=Ins|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=свой; word=своей; pos=DET; tag=Case=Ins|Gender=Fem|Number=Sing; score=0.9999>,
  <normal_form=жажда; word=жаждой; pos=NOUN; tag=Case=Ins|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=жизнь; word=жизни; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Sing; score=0.9308>,
  <normal_form=ян; word=Яна; pos=NOUN; tag=Case=Nom|Gender=Fem|Number=Sing; score=0.9873>,
  <normal_form=способный; word=способна; pos=ADJ; tag=Degree=Pos|Gender=Fem|Number=Sing|Variant=Short; score=0.9999>,
  <normal_form=испепелить; word=испеп

В данном случае "растопить" определилось верно. 
```
normal_form=растопить; word=растопить; pos=VERB; tag=VerbForm=Inf; score=1.0000>
```

In [102]:
sample_with_word(texts, tokenizer, predictor, 'топить')

1/1 [==============================] - 0s 47ms/step


/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


('Десять часов его станут топить в воде, жечь заживо, использовать в политических играх и угрожать санкциями.',
 [<normal_form=десять; word=Десять; pos=NUM; tag=Case=Acc; score=0.6233>,
  <normal_form=час; word=часов; pos=NOUN; tag=Case=Gen|Gender=Masc|Number=Plur; score=0.9999>,
  <normal_form=он; word=его; pos=PRON; tag=Case=Acc|Gender=Masc|Number=Sing|Person=3; score=0.5644>,
  <normal_form=стать; word=станут; pos=VERB; tag=Mood=Ind|Number=Plur|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act; score=1.0000>,
  <normal_form=топить; word=топить; pos=VERB; tag=VerbForm=Inf; score=1.0000>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=вода; word=воде; pos=NOUN; tag=Case=Loc|Gender=Fem|Number=Sing; score=0.9994>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=жечь; word=жечь; pos=VERB; tag=VerbForm=Inf; score=1.0000>,
  <normal_form=заживо; word=заживо; pos=ADV; tag=Degree=Pos; score=0.9996>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; sc

Тут приведено другое значение слова "топить", а именно в смысле утопления в воде. 
```
<normal_form=топить; word=топить; pos=VERB; tag=VerbForm=Inf; score=1.0000>
```

##### мир

In [127]:
sample_with_word(texts, tokenizer, predictor, 'мира')

1/1 [==============================] - 0s 84ms/step


/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


('Впрочем, для женщин здесь мало что изменилось за последние 300 лет, и юная Ада (Милана Агузарова), подпирающая стену на остановке маршрутки, отгородившись от мира безразмерной курткой и глухим воротом спортивной кофты, занята извечным женским делом — она ждет.',
 [<normal_form=впрочем; word=Впрочем; pos=H; tag=_; score=1.0000>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=для; word=для; pos=ADP; tag=_; score=1.0000>,
  <normal_form=женщина; word=женщин; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Plur; score=0.9979>,
  <normal_form=здесь; word=здесь; pos=ADV; tag=Degree=Pos; score=0.9996>,
  <normal_form=мало; word=мало; pos=PART; tag=_; score=0.9249>,
  <normal_form=что; word=что; pos=PRON; tag=Case=Nom|Gender=Neut|Number=Sing; score=0.9873>,
  <normal_form=измениться; word=изменилось; pos=VERB; tag=Gender=Neut|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Mid; score=1.0000>,
  <normal_form=за; word=за; pos=ADP; tag=_; score=1.0000>,
  <normal_form=пос

In [137]:
sample_with_word(texts, tokenizer, predictor, ' мир ')

1/1 [==============================] - 0s 105ms/step


/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


('\nНет нужды уточнять, соответствует ли этот взгляд действительности (подобного рода обобщения давно считаются неприличными, и, в общем, поделом); главное — что в нем чудится испуг пополам с восторгом, сложная, но едва ли безнадежно неприязненная эмоция, которая не слишком совпадает с образом России последних лет: фабрики троллей, тревожащего мир шатуна.',
 [<normal_form=
  ; word=
  ; pos=CONJ; tag=_; score=0.9772>,
  <normal_form=нет; word=Нет; pos=VERB; tag=Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin; score=0.9984>,
  <normal_form=нужда; word=нужды; pos=NOUN; tag=Case=Gen|Gender=Fem|Number=Sing; score=0.9972>,
  <normal_form=уточнять; word=уточнять; pos=VERB; tag=VerbForm=Inf; score=1.0000>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=соответствовать; word=соответствует; pos=VERB; tag=Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act; score=1.0000>,
  <normal_form=ли; word=ли; pos=PART; tag=_; score=1.0000>,
  <normal_form

#### Омоформы

##### Точно

In [140]:
sample_with_word(texts, tokenizer, predictor, ' точно ')

1/1 [==============================] - 0s 49ms/step


/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


('Почему эту яркую комедию точно не стоит пропускать и в чем ее главный месседж, рассказывает Юлия Салихова.',
 [<normal_form=почему; word=Почему; pos=ADV; tag=Degree=Pos; score=0.9992>,
  <normal_form=этот; word=эту; pos=DET; tag=Case=Acc|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=яркий; word=яркую; pos=ADJ; tag=Case=Acc|Degree=Pos|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=комедия; word=комедию; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=точно; word=точно; pos=CONJ; tag=_; score=0.5463>,
  <normal_form=не; word=не; pos=PART; tag=_; score=1.0000>,
  <normal_form=стоить; word=стоит; pos=VERB; tag=Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Act; score=1.0000>,
  <normal_form=пропускать; word=пропускать; pos=VERB; tag=VerbForm=Inf; score=1.0000>,
  <normal_form=и; word=и; pos=PART; tag=_; score=0.9407>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=что; word=чем; pos=PRON; tag=Case=Loc|Ge

Тут слово "точно" ошибочно определилось как союз, хотя в данном случае оно является наречием.
```
<normal_form=точно; word=точно; pos=CONJ; tag=_; score=0.5463>,
```

In [148]:
sample_with_word(texts, tokenizer, predictor, ' точно ')

1/1 [==============================] - 0s 68ms/step


/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


('Теперь все точно как в комиксах: не получится просто взять с полки «Удивительного Человека-Паука» под номером 700, не проштудировав предварительно хотя бы предыдущую сотню.',
 [<normal_form=теперь; word=Теперь; pos=ADV; tag=Degree=Pos; score=0.9997>,
  <normal_form=всё; word=все; pos=PRON; tag=Case=Nom|Gender=Neut|Number=Sing|Person=3; score=0.9921>,
  <normal_form=точный; word=точно; pos=ADJ; tag=Degree=Pos|Gender=Neut|Number=Sing|Variant=Short; score=0.9638>,
  <normal_form=как; word=как; pos=CONJ; tag=_; score=0.9977>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=комикс; word=комиксах; pos=NOUN; tag=Case=Loc|Gender=Masc|Number=Plur; score=0.9654>,
  <normal_form=:; word=:; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=не; word=не; pos=PART; tag=_; score=1.0000>,
  <normal_form=получиться; word=получится; pos=VERB; tag=Mood=Ind|Number=Sing|Person=3|Tense=Notpast|VerbForm=Fin|Voice=Mid; score=1.0000>,
  <normal_form=просто; word=просто; pos=ADV; tag=Degr

Тут снова неправильно определилось, но уже в обратном направлении. "Теперь все точно как в комиксах" - слово выступает как союз, а не прилагательное. Причем в данном случае модель показала 96% уверенности в своем прогнозе, что заставляет задуматься о ее точности. 
```
<normal_form=точный; word=точно; pos=ADJ; tag=Degree=Pos|Gender=Neut|Number=Sing|Variant=Short; score=0.9638>
```

##### дали

In [170]:
sample_with_word(texts, tokenizer, predictor, ' дали.')

/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


1/1 [==============================] - 0s 148ms/step


('\nОзону в этом идеальном teen-movie, которое он мечтал посмотреть 35 лет назад, да некому было его влажные поллюции перенести на экран, помогают любимые артисты старшего поколения: Мельвиль Пупо и Валери Бруни-Тедески — их роль сводится к оберегу, к весточке из родного дома, — а также убойный саундтрек, сплошь состоящий из хитов канувшей эпохи (тут The Cure и Род Стюарт с медляком Sailing обрамлены медоточивыми аккордами участника группы Air), ну и виды, конечно, безупречные атлантические дали.',
 [<normal_form=
  ; word=
  ; pos=CONJ; tag=_; score=0.9396>,
  <normal_form=озон; word=Озону; pos=NOUN; tag=Case=Dat|Gender=Masc|Number=Sing; score=0.9877>,
  <normal_form=в; word=в; pos=ADP; tag=_; score=1.0000>,
  <normal_form=этот; word=этом; pos=DET; tag=Case=Loc|Gender=Neut|Number=Sing; score=0.6496>,
  <normal_form=идеальный; word=идеальном; pos=ADJ; tag=Case=Loc|Degree=Pos|Gender=Neut|Number=Sing; score=0.7350>,
  <normal_form=teen; word=teen; pos=NOUN; tag=Case=Loc|Gender=Masc|Numbe

Тут "дали" ошибочно разобралось как глагол, хотя в данном случае является существительным. 
```
<normal_form=дать; word=дали; pos=VERB; tag=Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act; score=0.4727>
```

In [184]:
sample_with_word(texts, tokenizer, predictor, ' дали ')

/tmp/ipykernel_31/3980410749.py:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  sentence = np.random.choice(found_sentences)


1/1 [==============================] - 0s 124ms/step


('В их ослепительные дали иронически вложено замкнутое пространство военной части, где в вечном карауле, от наряда до дивана, тянется офицерская и супружеская лямка, «какая-то куцая, бескрылая жизнь, какая-то чепуха, и уйти и бежать нельзя», совершенно по Чехову или по Марко Беллоккьо, чей «Триумфальный марш» размазывал всю эту гротескную чепуху тяжелым армейским ботинком.',
 [<normal_form=в; word=В; pos=ADP; tag=_; score=1.0000>,
  <normal_form=он; word=их; pos=DET; tag=_; score=1.0000>,
  <normal_form=ослепительный; word=ослепительные; pos=ADJ; tag=Case=Acc|Degree=Pos|Number=Plur; score=0.9645>,
  <normal_form=дать; word=дали; pos=VERB; tag=Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act; score=0.7867>,
  <normal_form=иронически; word=иронически; pos=ADV; tag=Degree=Pos; score=1.0000>,
  <normal_form=вложенный; word=вложено; pos=ADJ; tag=Degree=Pos|Gender=Neut|Number=Sing|Variant=Short; score=0.9995>,
  <normal_form=замкнутый; word=замкнутое; pos=ADJ; tag=Case=Nom|Degree=Pos|G

Тут снова наблюдается такая же ошибка. 
```
<normal_form=дать; word=дали; pos=VERB; tag=Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act; score=0.7867>
```
Причем в обратную сторону в корпусе не допускается ошибок. Это говорит о том, что глаголы легче определять по их соотношению к существительному; также модели имеют предрасположенность при дисбалансе классов склоняться к тому, который чаще встречается (в данном случае глагол "дать" более популярен чем сущ. "даль").

### Недревесные явления

Тут уже будет легче искать явления в тексте, чем придумывать нетривиальные функция для поиска эллипсисов. 

In [153]:
def pipeline(sentence, tokenizer, predictor):
    sent = tokenizer(sentence)
    words = [token.text for token in sent]
    predictions = predictor.predict(words)
    return predictions

In [156]:
sentence = "Видимо, таков авторский вывод слегка непоследовательного, но всесторонне примечательного фильма «Проксима» — настоящее сестринство и материнство возможно, только если каждая занята своим делом (одна — покорением миров, вторая — просто учебой в школе), желательно — максимально далеко друг от друга."
sentence

'Видимо, таков авторский вывод слегка непоследовательного, но всесторонне примечательного фильма «Проксима» — настоящее сестринство и материнство возможно, только если каждая занята своим делом (одна — покорением миров, вторая — просто учебой в школе), желательно — максимально далеко друг от друга.'

In [155]:
sentence, pipeline(sentence, tokenizer, predictor)

1/1 [==============================] - 0s 90ms/step


('Видимо, таков авторский вывод слегка непоследовательного, но всесторонне примечательного фильма «Проксима» — настоящее сестринство и материнство возможно, только если каждая занята своим делом (одна — покорением миров, вторая — просто учебой в школе), желательно — максимально далеко друг от друга.',
 [<normal_form=видимо; word=Видимо; pos=ADV; tag=Degree=Pos; score=0.9999>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=таков; word=таков; pos=DET; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9940>,
  <normal_form=авторский; word=авторский; pos=ADJ; tag=Case=Nom|Degree=Pos|Gender=Masc|Number=Sing; score=0.9984>,
  <normal_form=вывод; word=вывод; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9996>,
  <normal_form=слегка; word=слегка; pos=ADV; tag=Degree=Pos; score=0.9619>,
  <normal_form=непоследовательный; word=непоследовательного; pos=ADJ; tag=Case=Gen|Degree=Pos|Gender=Masc|Number=Sing; score=0.9991>,
  <normal_form=,; word=,; pos=PUNCT; tag=_; 

Тут наблюдается множественная омонимия в конце предложения. В данном случае морфопарсер не допустил ошибок, при разборе данной части, так как части речи понятны и без контекста. 

In [157]:
sentence = "«Кто-нибудь видел мою девчонку» именно по этой траектории и развивается. Как эстетически, так и этически."
sentence

'«Кто-нибудь видел мою девчонку» именно по этой траектории и развивается. Как эстетически, так и этически.'

In [158]:
sentence, pipeline(sentence, tokenizer, predictor)

1/1 [==============================] - 0s 55ms/step


('«Кто-нибудь видел мою девчонку» именно по этой траектории и развивается. Как эстетически, так и этически.',
 [<normal_form=«; word=«; pos=PUNCT; tag=_; score=0.9429>,
  <normal_form=кто; word=Кто; pos=PRON; tag=Case=Nom|Number=Sing; score=1.0000>,
  <normal_form=-; word=-; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=нибыть; word=нибудь; pos=ADV; tag=Degree=Pos; score=0.8450>,
  <normal_form=видеть; word=видел; pos=VERB; tag=Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act; score=1.0000>,
  <normal_form=мой; word=мою; pos=DET; tag=Case=Acc|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=девчонка; word=девчонку; pos=NOUN; tag=Case=Acc|Gender=Fem|Number=Sing; score=1.0000>,
  <normal_form=»; word=»; pos=PUNCT; tag=_; score=0.9973>,
  <normal_form=именно; word=именно; pos=PART; tag=_; score=1.0000>,
  <normal_form=по; word=по; pos=ADP; tag=_; score=1.0000>,
  <normal_form=этот; word=этой; pos=DET; tag=Case=Dat|Gender=Fem|Number=Sing; score=0.9999>,
  <normal_f

### Референциальные цепочки

Но **Вигго Мортенсен** явно другой. **Он** вообще-то лицедей по совместительству. А так **поэт**, **художник**, **фотограф**, **куратор**, **полиглот**. **Символ** мультикультурализма и интерсекционализма. Внутри **Вигго**, как в **его** любимом Нью-Йорке, встречаются все семь искусств, чтобы фонтанировать без перебоя. **Его** режиссерский дебют «Падение», наверное, личная история с элементами автобиографии, но, как иногда случается у людей тонкой душевной конструкции, личное попало в глобальное, отразило дух времени, запечатлело уходящую эпоху.

In [160]:
fragment = "Но Вигго Мортенсен явно другой. Он вообще-то лицедей по совместительству. А так поэт, художник, фотограф, куратор, полиглот. Символ мультикультурализма и интерсекционализма. Внутри Вигго, как в его любимом Нью-Йорке, встречаются все семь искусств, чтобы фонтанировать без перебоя. Его режиссерский дебют «Падение», наверное, личная история с элементами автобиографии, но, как иногда случается у людей тонкой душевной конструкции, личное попало в глобальное, отразило дух времени, запечатлело уходящую эпоху."
fragment

'Но Вигго Мортенсен явно другой. Он вообще-то лицедей по совместительству. А так поэт, художник, фотограф, куратор, полиглот. Символ мультикультурализма и интерсекционализма. Внутри Вигго, как в его любимом Нью-Йорке, встречаются все семь искусств, чтобы фонтанировать без перебоя. Его режиссерский дебют «Падение», наверное, личная история с элементами автобиографии, но, как иногда случается у людей тонкой душевной конструкции, личное попало в глобальное, отразило дух времени, запечатлело уходящую эпоху.'

In [162]:
fragment, pipeline(fragment, tokenizer, predictor)

1/1 [==============================] - 0s 139ms/step


('Но Вигго Мортенсен явно другой. Он вообще-то лицедей по совместительству. А так поэт, художник, фотограф, куратор, полиглот. Символ мультикультурализма и интерсекционализма. Внутри Вигго, как в его любимом Нью-Йорке, встречаются все семь искусств, чтобы фонтанировать без перебоя. Его режиссерский дебют «Падение», наверное, личная история с элементами автобиографии, но, как иногда случается у людей тонкой душевной конструкции, личное попало в глобальное, отразило дух времени, запечатлело уходящую эпоху.',
 [<normal_form=но; word=Но; pos=CONJ; tag=_; score=1.0000>,
  <normal_form=вигй; word=Вигго; pos=ADJ; tag=Case=Acc|Degree=Pos|Gender=Masc|Number=Sing; score=0.5005>,
  <normal_form=мортенсный; word=Мортенсен; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9707>,
  <normal_form=явно; word=явно; pos=ADV; tag=Degree=Pos; score=0.9997>,
  <normal_form=другой; word=другой; pos=DET; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9352>,
  <normal_form=.; word=.; pos=PUNCT; tag=_; sc

Тут видно недочет у 2 элементов цепочки:
* Неправильная лемматизация имени "Вигго" и непоследовательная его лемматизация.
    
    <normal_form=вигй; word=Вигго; pos=ADJ; tag=Case=Acc|Degree=Pos|Gender=Masc|Number=Sing; score=0.5005>
    
    <normal_form=вигго; word=Вигго; pos=NOUN; tag=Case=Gen|Gender=Masc|Number=Sing; score=0.3481>
    
В остальном именно морфологический разбор проведен верно. Но, как мне кажется, морфопарсеры не подходят для выявления референциальных цепочек, так как это другой уровень разбора ЕЯ. 

**Персонаж** Майлса Теллера — это типичный религиозный аскет. В Испании **он** стал бы монахом, в России сектантом, хлыстом (прекрасная рифма к названию фильма!), а в протестантской Америке оказался почему-то барабанщиком. По ходу фильма **он** отказывается от мирских привязанностей, семейных, любовных, дружеских связей, добровольно проходит через чудовищные унижения и муки и терпит сокрушительное поражение, которое в финале, разумеется, оборачивается немыслимым триумфом. Беда в том, что Бога в **его** мире нет, и все что **он** делает, **он** делает не во Славу Божию.

In [163]:
fragment = "Персонаж Майлса Теллера — это типичный религиозный аскет. В Испании он стал бы монахом, в России сектантом, хлыстом (прекрасная рифма к названию фильма!), а в протестантской Америке оказался почему-то барабанщиком. По ходу фильма он отказывается от мирских привязанностей, семейных, любовных, дружеских связей, добровольно проходит через чудовищные унижения и муки и терпит сокрушительное поражение, которое в финале, разумеется, оборачивается немыслимым триумфом. Беда в том, что Бога в его мире нет, и все что он делает, он делает не во Славу Божию."
fragment

'Персонаж Майлса Теллера — это типичный религиозный аскет. В Испании он стал бы монахом, в России сектантом, хлыстом (прекрасная рифма к названию фильма!), а в протестантской Америке оказался почему-то барабанщиком. По ходу фильма он отказывается от мирских привязанностей, семейных, любовных, дружеских связей, добровольно проходит через чудовищные унижения и муки и терпит сокрушительное поражение, которое в финале, разумеется, оборачивается немыслимым триумфом. Беда в том, что Бога в его мире нет, и все что он делает, он делает не во Славу Божию.'

In [164]:
fragment, pipeline(fragment, tokenizer, predictor)

1/1 [==============================] - 0s 157ms/step


('Персонаж Майлса Теллера — это типичный религиозный аскет. В Испании он стал бы монахом, в России сектантом, хлыстом (прекрасная рифма к названию фильма!), а в протестантской Америке оказался почему-то барабанщиком. По ходу фильма он отказывается от мирских привязанностей, семейных, любовных, дружеских связей, добровольно проходит через чудовищные унижения и муки и терпит сокрушительное поражение, которое в финале, разумеется, оборачивается немыслимым триумфом. Беда в том, что Бога в его мире нет, и все что он делает, он делает не во Славу Божию.',
 [<normal_form=персонаж; word=Персонаж; pos=NOUN; tag=Case=Nom|Gender=Masc|Number=Sing; score=0.9975>,
  <normal_form=майлс; word=Майлса; pos=NOUN; tag=Case=Gen|Gender=Masc|Number=Sing; score=0.9999>,
  <normal_form=теллер; word=Теллера; pos=NOUN; tag=Case=Gen|Gender=Masc|Number=Sing; score=0.9882>,
  <normal_form=—; word=—; pos=PUNCT; tag=_; score=1.0000>,
  <normal_form=это; word=это; pos=PART; tag=_; score=0.9968>,
  <normal_form=типичны

Тут уже морфопарсинг прошел правильно в таком большом фрагменте текста, несмотря на эллипсис и референциальную цепочку. Это говорит о том, что данные явления редко являются препятствием для правильного мофрологического разбора. 